### Let's demonstrate the image formation pipeline in ``cryojax``.

In [ ]:
# Jax imports
import equinox as eqx
import jax
import jax.numpy as jnp
import numpy as np
from jax import config

config.update("jax_enable_x64", False)

In [ ]:
# Plotting imports and function definitions
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def plot_image(image, fig, ax, cmap="gray", label=None, **kwargs):
    im = ax.imshow(image, cmap=cmap, origin="lower", **kwargs)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(im, cax=cax)
    if label is not None:
        ax.set(title=label)
    return fig, ax

In [ ]:
# Image simulator imports
import cryojax.simulator as cs
from cryojax.utils import fftn, irfftn, fftfreqs

In [ ]:
# Volume filename and metadata
filename = "../tests/data/3jar_13pf_bfm1_ps5_28.mrc"

First we must read in our template into a given volume representation of the ``Specimen``. We also must choose its ``Pose``.

In [ ]:
# Read template into either an ElectronCloud or ElectronGrid
resolution = 5.28  # Angstroms
#density = cs.VoxelCloud.from_file(filename, config=dict(atol=1e-8))
density = cs.VoxelGrid.from_file(filename, config=dict(pad_scale=1.5))
pose = cs.EulerPose(offset_x=-50.0, offset_y=20.0, view_phi=-20, view_theta=100, view_psi=20)
specimen = cs.Specimen(density=density, pose=pose, resolution=resolution)

Next we have to choose a scattering (or in this case, projection) method. This will depend on the ``Specimen`` representation.

In [ ]:
# Configure the image formation process
shape = (81, 82)
pad_scale = 1.5
#scattering = cs.NufftScattering(shape=shape, pad_scale=pad_scale, eps=1e-5)
scattering = cs.FourierSliceScattering(shape=shape, pad_scale=pad_scale)

Now, it's time to configure the imaging instrument. This, at minimum, requires an ``Optics`` model. Here, we also include models of the ``Exposure`` to the electron beam, and the ``Detector``. For example, by default the optics model is ``NullOptics``. Here, we configure multiple instruments to demonstrate different stages of the image formation pipeline.

In [ ]:
# Initialize the instrument
optics = cs.CTFOptics(defocus_u=10000, defocus_v=10000, amplitude_contrast=.07)
exposure = cs.UniformExposure(N=1e5, mu=0.0)
detector = cs.GaussianDetector(pixel_size=resolution, key=jax.random.PRNGKey(seed=1234), variance=cs.Constant(1.0))
instrument_s = cs.Instrument(exposure=exposure)
instrument_o = cs.Instrument(exposure=exposure, optics=optics)
instrument_d = cs.Instrument(exposure=exposure, optics=optics, detector=detector)

Now, choose a model for the solvent. By default, this is just ``NullIce``. Here, we model the ice as gaussian colored noise with ``GaussianIce``.

In [ ]:
# Then, choose a model for the solvent
solvent = cs.GaussianIce(key=jax.random.PRNGKey(seed=0), variance=cs.Exp(amplitude=1.0, scale=1.0))

Finally, we can initialize our imaging model.

In [ ]:
# Image formation models
scattering_model = cs.ImagePipeline(scattering=scattering, specimen=specimen, instrument=instrument_s, solvent=solvent)
optics_model = cs.ImagePipeline(scattering=scattering, specimen=specimen, instrument=instrument_o, solvent=solvent)
detector_model = cs.ImagePipeline(scattering=scattering, specimen=specimen, instrument=instrument_d, solvent=solvent)

In [ ]:
# Plot models
fig, axes = plt.subplots(ncols=3, figsize=(12, 6))
ax1, ax2, ax3 = axes
im1 = plot_image(scattering_model(), fig, ax1, label="Exit plane")
im2 = plot_image(optics_model(), fig, ax2, label="Detector plane")
im3 = plot_image(detector_model(), fig, ax3, label="Detector readout")
plt.tight_layout()

Models can also include ``Filter``s. The following is a demonstration of a ``WhiteningFilter`` and a ``LowpassFilter``. These are precomputed upon initialization and applied automatically in the image formation pipeline. 

To compute a ``WhiteningFilter``, we first have to generate some synthetic data of structured noise from which we may compute its power spectrum.

In [ ]:
# Instantiate image filters
fig, axes = plt.subplot_mosaic("""
AAAABBB
""", figsize=(6.5, 3.5))
freqs = fftfreqs((800, 600), pixel_size=resolution)
micrograph = solvent.sample(freqs) * optics(freqs) + fftn(detector.sample(freqs))
whiten = cs.WhiteningFilter(scattering.shape, micrograph=micrograph)
plot_image(irfftn(micrograph.T), fig, axes["A"], label="Synthetic micrograph")
plot_image(whiten.filter, fig, axes["B"], label="Whitening filter")
plt.tight_layout()

Now generate an image with the toy icy structured noise model with a given whitening filter. We will also apply a low pass filter for antialiasing.

In [ ]:
# Show forward model for a whitened image
fig, ax = plt.subplots(figsize=(3.5, 3.5))
filters = [cs.LowpassFilter(scattering.padded_shape, cutoff=0.95), cs.WhiteningFilter(scattering.padded_shape, micrograph=micrograph)]
filtered_model = cs.ImagePipeline(scattering=scattering, specimen=specimen, instrument=instrument_d, filters=filters)
plot_image(filtered_model.render(), fig, ax)

If we want to solve the inverse problem and find the model parameters that most likely generated the data, we want to define a function that can be arbitrarily transformed by JAX and evaulated at subsets of the parameters.

In [ ]:
def update_model(pixel_size: jax.Array, model: cs.GaussianImage) -> cs.GaussianImage:
    return eqx.tree_at(lambda m: m.instrument.detector.pixel_size, model, pixel_size)

@jax.jit
def compute_image(pixel_size: jax.Array, model: cs.GaussianImage) -> jax.Array:
    model = update_model(pixel_size, model)
    return model()

In [ ]:
# Plot jitted model with updated parameters.
fig, axes = plt.subplots(ncols=2, figsize=(8, 6))
ax1, ax2 = axes
plot_image(compute_image(detector.pixel_size, detector_model), fig, ax1)
plot_image(compute_image(2*detector.pixel_size, detector_model), fig, ax2)

Now, we can also use the model to compute a likelihood. For this, we can use the ``GaussianImage`` model. This is a subclass of the ``ImagePipeline``, adding the ability to evaulate a likelihood. Let's evaulate the likelihood at the simulated data, and visualize the residuals.

In [ ]:
# Initialize the model and plot residuals
fig, axes = plt.subplots(ncols=3, figsize=(12, 6))
ax1, ax2, ax3 = axes
mask = cs.CircularMask(scattering.shape)
masks = [mask]
observation_model = cs.GaussianImage(scattering=scattering, specimen=specimen, instrument=instrument_d)
observed = mask(observation_model())
model = cs.GaussianImage(scattering=scattering, specimen=specimen, instrument=instrument_d, masks=masks, observed=observed)
simulated, observed, residuals = model.render(), model.observed, model.residuals
plot_image(simulated, fig, ax1)
plot_image(observed, fig, ax2)
plot_image(residuals, fig, ax3)
plt.tight_layout()

Evaluating the ``GaussianImage`` with the observed data set will automatically evaulate the ``GausianImage.log_likelihood`` function.

In [ ]:
def update_model(pixel_size: jax.Array, model: cs.GaussianImage) -> cs.GaussianImage:
    return eqx.tree_at(lambda m: m.instrument.detector.pixel_size, model, pixel_size)

@jax.jit
def compute_loss(pixel_size: jax.Array, model: cs.GaussianImage) -> jax.Array:
    model = update_model(pixel_size, model)
    return model()

@jax.jit
@jax.grad
def compute_grad(pixel_size: jax.Array, model: cs.GaussianImage) -> jax.Array:
    return compute_loss(pixel_size, model)

In [ ]:
# Benchmark loss pipeline
%timeit likelihood = compute_loss(detector.pixel_size+.01, model)

In [ ]:
# Benchmark gradient pipeline
%timeit grad = compute_grad(detector.pixel_size+.01, model)